In [ ]:
# down forget to upload abcd file to the source file in here, chose 'upload the session storage' *
# and download abcd file to your computer, it is right in the folder too *

import json
import pandas as pd
from pandas import json_normalize
import tensorflow as tf 

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

import os
import pandas as pd
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

import re
import preprocessor as p

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from wordcloud import WordCloud



In [ ]:
with open('abcd.json','r') as f:
 data = json.loads(f.read())

In [ ]:
data_train = data['train']
data_dev = data['dev']
data_test = data['test']


In [ ]:
df_20 = pd.json_normalize(data_train, record_path=['delexed'], meta = [['convo_id'],['scenario','personal'],['scenario','order'],['scenario','product'],['scenario','flow'],['scenario','subflow']])

In [ ]:
df_20.columns
df_20.shape[0] 

176434

In [ ]:
#df_20 = df_train.sample(n = 35286, random_state = 2)
#df_20 = df_20.sample(n = 5000, random_state = 5)
#df_20.sample(frac=0.2, replace=True, random_state=1)

In [ ]:
df_20
colnames_to_drop = ['turn_count','targets', 'candidates', 'scenario.order', 'scenario.product', 'scenario.personal','scenario.subflow'] 
df3 = pd.DataFrame(df_20.drop(colnames_to_drop, axis=1))


In [ ]:

df3


,speaker,text,convo_id,scenario.flow
0,agent,hi!,3592,product_defect
1,agent,how can i help you?,3592,product_defect
2,customer,"hi! i need to return an item, can you help me ...",3592,product_defect
3,agent,"sure, may i have your name please?",3592,product_defect
4,customer,crystal minh,3592,product_defect
...,...,...,...,...
176429,customer,ok that seemed to be the problem,5882,troubleshoot_site
176430,customer,thank you,5882,troubleshoot_site
176431,agent,is there anything else i can help with today,5882,troubleshoot_site
176432,customer,that is all,5882,troubleshoot_site


In [ ]:
df1 = df3[df3['speaker'] == 'agent']
df1
df2 = df3[df3['speaker'] == 'customer']
df2
df1.reset_index(drop=True, inplace=True) #reset the index
df2.reset_index(drop=True, inplace=True)
df4 = pd.merge( df1, df2, left_index=True, right_index=True, how='outer', suffixes=('_agent', '_customer'))
df4.head(4)


,speaker_agent,text_agent,convo_id_agent,scenario.flow_agent,speaker_customer,text_customer,convo_id_customer,scenario.flow_customer
0,agent,hi!,3592,product_defect,customer,"hi! i need to return an item, can you help me ...",3592,product_defect
1,agent,how can i help you?,3592,product_defect,customer,crystal minh,3592,product_defect
2,agent,"sure, may i have your name please?",3592,product_defect,customer,i got the wrong size.,3592,product_defect
3,agent,"thanks, may i ask the reason for the return?",3592,product_defect,customer,username: <username>,3592,product_defect


In [ ]:
# renaming columns flow to the 'category'
df4 = df4.rename(columns={'scenario.flow_agent':"category"})


In [ ]:
df4

,speaker_agent,text_agent,convo_id_agent,category,speaker_customer,text_customer,convo_id_customer,scenario.flow_customer
0,agent,hi!,3592,product_defect,customer,"hi! i need to return an item, can you help me ...",3592,product_defect
1,agent,how can i help you?,3592,product_defect,customer,crystal minh,3592,product_defect
2,agent,"sure, may i have your name please?",3592,product_defect,customer,i got the wrong size.,3592,product_defect
3,agent,"thanks, may i ask the reason for the return?",3592,product_defect,customer,username: <username>,3592,product_defect
4,agent,"ok, may i have your username, email address an...",3592,product_defect,customer,<email>,3592,product_defect
...,...,...,...,...,...,...,...,...
75980,agent,"first, try logging in and out of your account,...",5882,troubleshoot_site,NaN,NaN,NaN,NaN
75981,agent,"if that doesn't work, go ahead and clear your ...",5882,troubleshoot_site,NaN,NaN,NaN,NaN
75982,agent,should be in your browser settings under more ...,5882,troubleshoot_site,NaN,NaN,NaN,NaN
75983,agent,is there anything else i can help with today,5882,troubleshoot_site,NaN,NaN,NaN,NaN


In [ ]:
# Shuffel the dataset to make sure we get an equal distribution of the data before splitting into train and test sets
#dataset = df.sample(frac=1)
#dataset.head(4)

In [ ]:
sorted = df4.groupby('category')
  
# Let's print the first entries
# in all the groups formed.
sorted.first()

,speaker_agent,text_agent,convo_id_agent,speaker_customer,text_customer,convo_id_customer,scenario.flow_customer
category,,,,,,,
account_access,agent,hello how can i help you today?,8893,customer,"hello, i am trying to check the status of my o...",8893,account_access
manage_account,agent,thanks for choosing acmebrands! what can i do ...,2474,customer,sure. it is alessandro phoenix and my acct id...,974,order_issue
order_issue,agent,hi! how can i help you today?,8080,customer,thank you!,10410,troubleshoot_site
product_defect,agent,hi!,3592,customer,"hi! i need to return an item, can you help me ...",3592,product_defect
purchase_dispute,agent,"good afternoon, how can i help you?",1699,customer,"all right, thanks for your help",8080,order_issue
shipping_issue,agent,welcome to acmebrands! how can i help you?,5798,customer,yes my order id is <order_id>,5798,shipping_issue
single_item_query,agent,"hello, it is not dry clean only. we use high q...",1390,customer,"it says ""in transit",1884,shipping_issue
storewide_query,agent,"good afternoon, how can i help you?",3695,customer,how much long till it is refunded,9489,product_defect
subscription_inquiry,agent,hello! how can i help you today?,1426,customer,i want an extension for my shopping service.,1426,subscription_inquiry


In [ ]:
# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)


def clean_text(df4):
    #set up punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    tempArr = []
    for line in df4:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove puctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [ ]:
# Cleaning text for Agent 

# cleaning the text
df4['clean _text_agent'] = clean_text(df4['text_agent'])
df4.head()

,speaker_agent,text_agent,convo_id_agent,category,speaker_customer,text_customer,convo_id_customer,scenario.flow_customer,clean _text_agent
0,agent,hi!,3592,product_defect,customer,"hi! i need to return an item, can you help me ...",3592,product_defect,hi
1,agent,how can i help you?,3592,product_defect,customer,crystal minh,3592,product_defect,how can i help you
2,agent,"sure, may i have your name please?",3592,product_defect,customer,i got the wrong size.,3592,product_defect,sure may i have your name please
3,agent,"thanks, may i ask the reason for the return?",3592,product_defect,customer,username: <username>,3592,product_defect,thanks may i ask the reason for the return
4,agent,"ok, may i have your username, email address an...",3592,product_defect,customer,<email>,3592,product_defect,ok may i have your username email address and ...


In [ ]:
corpus = list(df4['clean _text_agent'])
corpus = corpus[0:500]
# not enough of examples 

In [ ]:
corpus_embeddings = embedder.encode(corpus)

In [ ]:
num_clusters = 435
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [ ]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df.head()

,corpus,cluster
0,hi,361
1,how can i help you,391
2,sure may i have your name please,369
3,thanks may i ask the reason for the return,113
4,ok may i have your username email address and ...,405


In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")
   

Cluster  1
['to validate the purchase can i also get your username email and order id']

Cluster  2
['okay let me see what i can do to help', 'okay let me see what i can do to help']

Cluster  3
['awesome have a great night']

Cluster  4
['sure give me one moment']

Cluster  5
['thank you', 'thank you', 'thank you', 'thank you', 'thank you']

Cluster  6
['have a good day', 'have a good day', 'have a good day', 'have a good day']

Cluster  7
['did that help answer your question']

Cluster  8
['is there anything else i can help you with today', 'is there anything else i can help you with today', 'is there anything else i can help you with today']

Cluster  9
['when did you purchase the item', 'when did you purchase the item']

Cluster  10
['oh im sorry to hear that but i should be able to update your information']

Cluster  11
['okay i can help with that could i have your full name please']

Cluster  12
['hello thank you for contacting acmebrands how can i help you today']

Cluster  13
[

In [ ]:
#cluster_df['corpus'].rows()

In [ ]:

df5 = pd.merge( df4, cluster_df, left_index=True, right_index=True, how='outer', suffixes=('_agent', '_customer'))
df5.head(5)


,speaker_agent,text_agent,convo_id_agent,category,speaker_customer,text_customer,convo_id_customer,scenario.flow_customer,clean _text_agent,corpus,cluster
0,agent,hi!,3592,product_defect,customer,"hi! i need to return an item, can you help me ...",3592,product_defect,hi,hi,361.0
1,agent,how can i help you?,3592,product_defect,customer,crystal minh,3592,product_defect,how can i help you,how can i help you,391.0
2,agent,"sure, may i have your name please?",3592,product_defect,customer,i got the wrong size.,3592,product_defect,sure may i have your name please,sure may i have your name please,369.0
3,agent,"thanks, may i ask the reason for the return?",3592,product_defect,customer,username: <username>,3592,product_defect,thanks may i ask the reason for the return,thanks may i ask the reason for the return,113.0
4,agent,"ok, may i have your username, email address an...",3592,product_defect,customer,<email>,3592,product_defect,ok may i have your username email address and ...,ok may i have your username email address and ...,405.0


In [ ]:

df5['clean _text_agent_corpus'] = clean_text(df5['text_agent'])
df5.head()

,speaker_agent,text_agent,convo_id_agent,category,speaker_customer,text_customer,convo_id_customer,scenario.flow_customer,clean _text_agent,corpus,cluster,clean _text_agent_corpus
0,agent,hi!,3592,product_defect,customer,"hi! i need to return an item, can you help me ...",3592,product_defect,hi,hi,361.0,hi
1,agent,how can i help you?,3592,product_defect,customer,crystal minh,3592,product_defect,how can i help you,how can i help you,391.0,how can i help you
2,agent,"sure, may i have your name please?",3592,product_defect,customer,i got the wrong size.,3592,product_defect,sure may i have your name please,sure may i have your name please,369.0,sure may i have your name please
3,agent,"thanks, may i ask the reason for the return?",3592,product_defect,customer,username: <username>,3592,product_defect,thanks may i ask the reason for the return,thanks may i ask the reason for the return,113.0,thanks may i ask the reason for the return
4,agent,"ok, may i have your username, email address an...",3592,product_defect,customer,<email>,3592,product_defect,ok may i have your username email address and ...,ok may i have your username email address and ...,405.0,ok may i have your username email address and ...


In [ ]:
#Logistic Regression 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
def clean_text(df5):
    #set up punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    tempArr = []
    for line in df4:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove puctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [ ]:

def clean_dataset(df5):
    assert isinstance(df5, pd.DataFrame), "df needs to be a pd.DataFrame"
    df5.dropna(inplace=True)
    indices_to_keep = ~df5.isin([np.nan, np.inf, -np.inf]).any(1)
    return df5[indices_to_keep].astype(np.float64)

In [ ]:
from sklearn.model_selection import train_test_split
cleaned_text =df5['clean _text_agent_corpus']
cleaned_text = np.nan_to_num(cleaned_text)
agent_cluster = df5['cluster']
cleaned_text = np.nan_to_num(agent_cluster)
x_train, x_test, y_train, y_test = train_test_split(cleaned_text, agent_cluster, test_size=0.20, random_state=42)
x_train, x_cv, y_train, y_cv = train_test_split(x_train,y_train, test_size=0.20, random_state=42)

# features - C , Q
# label - Answers 
# Confusion MAtrix 


In [ ]:
"""
test_features = transformer.transform(["hi! i need to return an item, can you help me "])
model.predict(test_features)
get_top_k_predicitons(model,test_features,2)
"""

In [ ]:
#it will convert the text into the numerical vector
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(x_train)

X_train = count_vect.transform(x_train)
X_cv = count_vect.transform(x_cv)
X_test = count_vect.transform(x_test)

AttributeError: ignored

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression as MNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score as AUC

In [ ]:

alpha = [.0001,.001,.01,.1,1,10,100]
alpha1 = np.log(alpha)
roc_auc_cv = []
roc_auc_train = []
opt_c = 0
opt_auc = 0
for i in alpha:
    nv =MNB(C=i,penalty = 'l2')
    nv.fit(X_train,y_train)
    prediction1 = nv.predict_proba(X_cv)[:,1]
    score1 = AUC(X_cv,prediction1)
    prediction2 = nv.predict_proba(X_train)[:,1]
    score2 = AUC(y_train,prediction2)
    print(i," ------> ",score1)
    roc_auc_train.append(score2)
    roc_auc_cv.append(score1)
    if(opt_auc<score1):
        opt_auc = score1
        opt_c = i
print("Optimal AUC score:",opt_auc)
print("Optimal C:",opt_c)

ValueError: ignored

In [ ]:
# Take 3 values of cluster based on cluster 